# Trabalho prático II - Classificação

O objetivo deste trabalho é praticar os conceitos de aprendizado supervisionado que vimos em sala.

A sua tarefa será treinar um classificador para um conjunto de dados misterioso (se eu falasses qual é o objetivo do modelo, você encontraria soluções na Internet).

Baixe o arquivo [t2.tar.gz](https://drive.google.com/file/d/125plHKUzFGxHjjCiVJcTQG2bPG_zgDNV/view?usp=sharing). Descoprima este arquivo para encontrar outros quatro. Os arquivos `train_X.csv` e `train_y.csv` possuem os dados que você deve usar para treinar o modelo. O arquivo `test_X.csv` possui os objetos para os quais você deve encontrar as classes (testar o modelo). Por fim, o arquivo `test_example_y.csv` é um exemplo de como sua solução final deve ser organizada.

Cada linha dos arquivos `train_X.csv` e `test_X.csv` tem 15 campos descrevendo um objeto misterioso. O campo `id` representa o identificador do objeto, sendo que este campo não deve ser considerado em seu modelo. Os atributos a serem usados no modelo são os 14 campos rotulados de de `a` até `n`. Desses atributos:
- `b`, `d`, `f`, `g`, `h`, `i`, `j` e `n` são categóricos; e
- `a`, `c`, `e`, `k`, `l` e `m` são numéricos.

Cada linha do arquivo `train_y.csv` possui dois campos. O primeiro é o identificador de um objeto do arquivo `train_X.csv` e o segundo é a classe do respectivo objeto (0 ou 1).

Seu objetivo é encontrar as classes dos objetos do arquivo `test_X.csv` e mostrar como chegou em sua solução! Os dados do arquivo de teste foram obtido a partir de uma amostra aleatório do todo. Ou seja, um modelo bem treinado, e sem _overfitting_, em `train_X.csv` e `train_y.csv` se sairá bem em `test_X.csv`.

**Data de entrega:** dia 4 de julho de 2018.

**Grupo:** de até 3 pessoas, mas duas pessoas do mesmo grupo no trabalho 1 não podem pertencer ao mesmo grupo nesse trabalho.

**Valor:** 20% da nota do semestre.

Os três seguintes pontos descrevem o que obrigatoriamente deve ser entregue, com seu respectivo valor.

1 - **[10 pontos]** Este notebook com todo seu código e resultados (números, tabelas e gráficos). Você pode usar qualquer um dos métodos que estudamos ou alguma de suas variações próximas. Se estiver na dúvida se pode usar um método, basta perguntas no Piazza. Comentários e justificativas no notebook não serão considerados para sua nota.
O notebook deve ser enviado para o email do professor.

2 - **[8 pontos]** Um relatório digitado contendo: capa, introdução, metodologia, resultados, conclusão e referências. O relatório deve ter no máximo 10 páginas, com coluna simples, fonte 11, espaçamento 1.5 e margens de 2cm. A seção de metodologia deve conter uma descrição detalhada dos passos seguidos (não incluir código no relatório). A seção de resultados deve conter obrigatoriamente: uma caracterização descritiva dos dados, matriz de confusão das predições, _precision_, _recall_, _F1 score_ e acurácia. Todas as métricas de predição devem ser calculadas a partir dos arquivos de treinamento por meio de validação cruzada.
O relatório deve ser enviado para o email do professor.

3 - **[2 pontos + equivalente a lista extra pela classificação]** A sua predição final do arquivo `test_X.csv` deve ser enviada para o professor por email. O formato deve ser o mesmo do arquivo `train_y.csv`, assim como exemplificado em `test_example_y.csv` (mas repare que as classes desse último arquivo foram gerados de forma aleatória). Em outras palavras, o arquivo a ser entregue deve ter dois campos. O campo `id` é o identificador do objeto em `test_X.csv` e o campo `label` é a classe que seu modelo encontrou para o objeto em questão. A primeira linha do arquivo deve conter os nomes das colunas.
A entrega desse arquivo é obrigatória e vale dois pontos. Além disso, o trabalho com maior _F1 score_ ganhará o equivalente a 100% de uma lista extra. O trabalho com o pior _F1 score_ não ganhará nota extra alguma. Os demais trabalhos terão nota proporcional.
O professor se reserva o direito de anular esse quesito (nota extra) se houver indícios de má conduta durante a competição.

**Kaggle:** Estou tentando criar uma competição para esse trabalho na plataforma _Kaggle_. Se eu conseguir, compartilho o _link_ com você no _Piazza_.

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import preprocessing as pp
from sklearn import metrics as mt
from sklearn import model_selection as ms

In [2]:
#leitura dos dados
train_X = pd.read_csv("train_X.csv")
train_Y = pd.read_csv("train_y.csv")
x = pd.read_csv("test_X.csv")
y = pd.read_csv("test_example_y.csv")

In [3]:
#Ajuste dos dados (binariza dados categóricos no dataset)
#b, d, f, g, h, i, j e n são categóricos
#a, c, e, k, l e m são numéricos
def binarizeData(data, columns):
    binData = data.drop(columns, axis=1)
    for col in columns:
        l = pp.LabelBinarizer()
        auxData = pd.DataFrame(l.fit_transform(data[col]))
        colNames = [col + '_' + str(i) for i in range(len(auxData.columns))]
        auxData.columns = colNames
        binData[colNames] = auxData
    return binData

#train_X --> 40222 rows × 15 columns
#x --> 5000 rows × 15 columns
dataFrame = pd.concat([train_X, x])
dataFrame_bin = binarizeData(dataFrame, ["b","d","f","g","h","i","j","n"])

#Normalizando os dados
scaler = pp.MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(dataFrame_bin.loc[:,'a':]))

#Definição dos conjuntos
baseTrain = data.loc[:40221,:]
predict = data.loc[40222:,:]

In [4]:
classe = train_Y.loc[:,'label':]
train_x, test_x, train_y, test_y = ms.train_test_split(baseTrain, classe["label"].values, test_size=0.25, random_state=33)

## Aplicação dos metodos

In [12]:
#Treinando os dados (usando SVM)
clf1 = svm.SVC(kernel="rbf", C=0.1, degree=2).fit(train_x, train_y)

In [14]:
from sklearn import tree 
#Treinando os dados (usando Árvore de decisão)
clf2 = tree.DecisionTreeClassifier().fit(train_x, train_y)

In [5]:
from sklearn import ensemble as ens
#Treinando os dados (usando Florestas aleatórias)
clf3 = ens.RandomForestClassifier(n_estimators=10000, criterion='entropy', warm_start=True, n_jobs=-1).fit(train_x, train_y)

In [30]:
from sklearn import naive_bayes as nb
#Treinando os dados (usando Naive Bayes)
clf4 = nb.BernoulliNB(alpha=0.01, binarize=0.4, class_prior=None, fit_prior=True).fit(train_x, train_y)

In [31]:
#Predizendo a classe do conjunto de teste
def score(clf, pred, target):
    predictData = clf.predict(pred)
    return ("precision: "+str('{:.5f}'.format(mt.precision_score(target, predictData))), "recall: "+str('{:.5f}'.format(mt.recall_score(target, predictData))), "f1_score: "+str('{:.5f}'.format(mt.f1_score(target, predictData))), "accuracy: "+str('{:.5f}'.format(mt.accuracy_score(target, predictData))))

print(score(clf4, test_x, test_y))

#Plot da matriz de confusão
#Classificadores disponiveis: {clf(x): 1 <= x <= 4}
#1-SVM; 2-decision tree; 3-random forest; 4-naive bayes.
print("\nMatriz de Confusão:\n")
print(mt.confusion_matrix(train_y, clf4.predict(train_x)))

('precision: 0.53078', 'recall: 0.79231', 'f1_score: 0.63570', 'accuracy: 0.77695')

Matriz de Confusão:

[[17599  5064]
 [ 1599  5904]]


### Tabela de Resultados:

|Método       |Score (predição, precision, recall, F1 score e acurácia)             |
|-------------|---------------------------------------------------------------------|
|svm          |(0.71990, 0.47449, 0.57199, 0.82558)                                 |
|decision tree|(0.61341, 0.62955, 0.62138, 0.81156)                                 |
|random forest|(0.74017, 0.64008, 0.68650, 0.85640)                                 |
|naive bayes  |(0.53078, 0.79231, 0.63570, 0.77695)                                 |

## Geração do arquivo para envio

In [32]:
pdId = x["id"]
pdData = pd.DataFrame(clf3.predict(predict), columns=["label"])
send = pd.concat([pdId, pdData], axis=1)
send.to_csv("81814_Solution.csv")